In [ ]:
pip install pynput

In [ ]:
import mmap
import struct
import time
from pynput.keyboard import Key, Listener, KeyCode
import threading

# Define the struct format
struct_format = "fff???"  # Three floats and a boolean


g_start = False
g_movex = False
g_movey = False
g_movez = False

def mainloop():
    print("mainloop")
    global g_start
    global g_movex
    global g_movey
    global g_movez
    n = 0
    x = 0.5
    y = 1
    z = -0.4
    left = True
    yadd = True
    zadd = True
    while n < 10000: 
        if not g_start:
            continue;
            
        if g_movex:
            if x < -1:
                left = False
            elif x > 2:
                left = True
        
            if left:
                x = x - 0.01
            else:
                x = x + 0.01

        if g_movey:
            if y < -1:
                yadd = True
            elif y > 2:
                yadd = False

            if yadd:
                y = y + 0.01
            else:
                y = y - 0.01

        if g_movez:
            if z < -2:
                zadd = True
            elif z > 2:
                zadd = False

            if zadd:
                z = z + 0.01
            else:
                z = z - 0.01
                
        print(f"x: {x}, y:{y}, z:{z}")
        # Define the data to be written to shared memory
        data = (x, y, z, False, False, False )
        
        # Pack the data into a byte string
        packed_data = struct.pack(struct_format, *data)
        
        # Open or create a named shared memory region
        shm = mmap.mmap(-1, struct.calcsize(struct_format), "DriverSharedMemory")
        
        # Write the packed data to shared memory
        shm.write(packed_data)
        time.sleep(0.1)
        n += 1
        
    
def on_press(key):
    global g_start
    global g_movex
    global g_movey
    global g_movez
    if key == Key.enter:
        print("Enter key is pressed.")
        g_start = not g_start
    if key == KeyCode.from_char('q'):
        print("x key is pressed.")
        g_movex = not g_movex
    if key == KeyCode.from_char('w'):
        print("y key is pressed.")
        g_movey = not g_movey
    if key == KeyCode.from_char('e'):
        print("z key is pressed.")
        g_movez = not g_movez

def start_listener():
    with Listener(on_press=on_press) as listener:
        print("start listener")
        listener.join()

# Create a thread for the main loop
mainloop_thread = threading.Thread(target=mainloop)

# Create a thread for the listener
listener_thread = threading.Thread(target=start_listener)

# Start the main loop thread
mainloop_thread.start()

# Start the listener thread
listener_thread.start()

# Wait for both threads to finish
mainloop_thread.join()
listener_thread.join()
# Close the shared memory region
shm.close()




In [ ]:




option: x:0.02999     y:0.849999      z:-0.4
edit:   x:0.02999     y:0.579999      z:-0.4
inputfield: x:-0.290  y:1.15 z:-0.4   shift:

x: -1.810000 y:0.900000 z:-1.650000, pos.x:-2.199023 pos.y:1.548772 pos.z:-1.753361
x: 1.940000 y:-0.150000 z:0.000000, pos.x:1.969107 pos.y:0.913113 pos.z:-0.055828
x: 0.14000000000000126, y:1.5, z:0
x: 0.14000000000000126, y:1.05, z:0
x: 0.14000000000000126, y:0.6000000000000002, z:0
pos.x:0.050000 pos.y:0.700000 pos.z:-0.400000




In [ ]:
import mmap
import struct
import time
from pynput.keyboard import Key, Listener, KeyCode
import threading
import json

# 读取 JSON 文件
with open('testui.json', 'r') as file:
    data = json.load(file)

# Define the struct format
struct_format = "fff???"  # Three floats and tree boolean

g_start = False

class GameObject:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def movetoposition(self, target_x, target_y, target_z, duration, bclick, bdown, bup, sendsharememory):
        start_time = time.time()
        end_time = start_time + duration
        
        start_x, start_y, start_z = self.x, self.y, self.z
        
        while time.time() <= end_time:
            current_time = time.time()
            progress = (current_time - start_time) / duration
            
            # Calculate the intermediate position using linear interpolation
            self.x = start_x + (target_x - start_x) * progress
            self.y = start_y + (target_y - start_y) * progress
            self.z = start_z + (target_z - start_z) * progress
            
            print(f"Position: ({self.x}, {self.y}, {self.z})")
            sendsharememory(self.x, self.y, self.z, False, False, False)
            
            # Adjust this value for smoother movement
            time.sleep(0.01)
        self.x, self.y,self.z = target_x,target_y,target_z
        sendsharememory(self.x, self.y, self.z, bclick, bdown, bup)
        # sendsharememory(self.x, self.y, self.z, bclick, bdown, bup)
            
def sendsharememory(x,y,z, bclick,bdown,bup):
    # Define the data to be written to shared memory
    data = (x, y, z, bclick,bdown,bup)
    
    # Pack the data into a byte string
    packed_data = struct.pack(struct_format, *data)
    
    # Open or create a named shared memory region
    shm = mmap.mmap(-1, struct.calcsize(struct_format), "DriverSharedMemory")
    
    # Write the packed data to shared memory
    shm.write(packed_data)
    time.sleep(0.1)
    # Close the shared memory region
    # shm.close()

def mainloop():
    print("mainloop")
    global g_start

    obj = GameObject(0, 0, 0)
    while True:
        if g_start :
            for dataset in data:
                x = dataset["x"]
                y = dataset["y"]
                z = dataset["z"]
                ftime = dataset["ftime"]
                bclick = dataset["bclick"]
                bdown = dataset["bdown"]
                bup = dataset["bup"]
                obj.movetoposition(x,y,z,ftime,bclick,bdown,bup, sendsharememory)

def on_press(key):
    global g_start

    if key == Key.enter:
        print("Enter key is pressed.")
        g_start = not g_start

def start_listener():
    with Listener(on_press=on_press) as listener:
        print("start listener")
        listener.join()

# Create a thread for the main loop
mainloop_thread = threading.Thread(target=mainloop)

# Create a thread for the listener
listener_thread = threading.Thread(target=start_listener)

# Start the main loop thread
mainloop_thread.start()

# Start the listener thread
listener_thread.start()

# Wait for both threads to finish
mainloop_thread.join()
listener_thread.join()



